# Tutoriel URBasic

Set the comunication with the robot

In [ ]:
import sys
import numpy as np
from time import sleep

from URBasic import ISCoin
from URBasic import Joint6D
from URBasic import TCP6D
from URBasic import TCP6DDescriptor

robot_ip = ""
robot_opened_gripper_size_mm = 40. # Max oppenning of gripper

iscoin = ISCoin(robot_ip, robot_opened_gripper_size_mm)

if not iscoin.isInitOk:
    print("Error robot not initialised")
    sys.exit(1)

# Base robot position in Joint space
j_o = Joint6D.createFromRadians(
        -1.5708, -1.5708, -1.0472, -2.0944, 1.5708, 0.0
    )

## TCP Calibration

### Collect TCP pivot data

In [ ]:
iscoin.robot_control.reset_error()

# initial_pos   = TCP6D.createFromMetersRadians(x, y, z, rx, ry, rz)    # Coordinate of the tools
# initial_joint = Joint6D.createFromRadians(j1, j2, j3, j4, j5, j6)     # Value of the robot joints

robot_arm = iscoin.robot_control

robot_arm.freedrive_mode()

tcps = []

NUM_MEASURES = 20

for k in range(NUM_MEASURES):
    robot_arm.freedrive_mode()
    print(f"[{k+1}/{NUM_MEASURES}] Move robot, then press ENTER to capture. (or q to quit)")
    i = input()
    if i == "q":
        if k < 6:
            print("You should give min 6 points")
            continue
        else:
            break

    robot_arm.end_freedrive_mode()

    # Return robot positions
    tcp = robot_arm.get_actual_tcp_pose().toList()

    tcps.append(tcp)
    print("Captured sample", k+1)


robot_arm.end_freedrive_mode()

### Pivot calibration

In [ ]:
def pose_to_RT(pose):
    # pose = [x, y, z, rx, ry, rz] (UR format)
    x, y, z, rx, ry, rz = pose
    theta = np.linalg.norm([rx, ry, rz])
    if theta < 1e-9:
        R = np.eye(3)
    else:
        k = np.array([rx, ry, rz]) / theta
        K = np.array([[0, -k[2], k[1]],
                      [k[2], 0, -k[0]],
                      [-k[1], k[0], 0]])
        R = np.eye(3) + np.sin(theta)*K + (1-np.cos(theta))*(K@K)
    t = np.array([x, y, z])
    return R, t

def solve_tcp(tcps):
    A = []
    b = []

    for pose in tcps:
        R, t = pose_to_RT(pose)
        A.append(np.hstack([R, -np.eye(3)]))
        b.append(-t)

    A = np.vstack(A)
    b = np.hstack(b)

    x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

    p = x[:3]   # TCP in flange frame
    P = x[3:]   # world point (not needed)

    return p, P

# After your data collection:
tcp_vector, world_point = solve_tcp(tcps)

print("TCP (flange frame):", tcp_vector)

tcp_offset = TCP6D.createFromMetersRadians(tcp_vector[0], tcp_vector[1], tcp_vector[2], 0 , 0, 0)
robot_arm.set_tcp(tcp_offset)

### Validation Calibration

In [ ]:
# Capture reference pose
robot_arm.freedrive_mode()
input("Move robot so the stylus is in the hole, then press ENTER.")
tcp_ref = robot_arm.get_actual_tcp_pose()
robot_arm.end_freedrive_mode()

# Move back to reference pose once
robot_arm.movel(tcp_ref, a=1.2, v=0.25)

ROT = 0.5   # about ~28 degrees

# Helper: apply rotation around TCP
def rotate_around_tcp(tcp: TCP6D, rx, ry, rz):
    new_pose = TCP6D.createFromMetersRadians(
        tcp.x, tcp.y, tcp.z,
        tcp.rx + rx,
        tcp.ry + ry,
        tcp.rz + rz
    )
    return new_pose

# Perform validation rotations
axes = [
    (ROT, 0, 0),
    (-ROT, 0, 0),
    (0, ROT, 0),
    (0, -ROT, 0),
    (0, 0, ROT),
    (0, 0, -ROT)
]

for rx, ry, rz in axes:
    test_pose = rotate_around_tcp(tcp_ref, rx, ry, rz)
    print(f"Rotating: rx={rx}, ry={ry}, rz={rz}")

    robot_arm.movel(test_pose, a=1.2, v=0.25)
    robot_arm.waitRobotIdleOrStopFlag()
    sleep(2)
    robot_arm.movel(tcp_ref, a=1.2, v=0.25)
    robot_arm.waitRobotIdleOrStopFlag()
    sleep(1)

print("Validation complete.")


### Relatif TCP detection

In [ ]:
def similarity_transform_3D(A, B):
    """
    Compute s, R, t such that:  B ≈ s * R @ A + t
    A: Nx3 points in source frame
    B: Nx3 points in target frame
    """
    assert A.shape == B.shape
    N = A.shape[0]

    centroid_A = A.mean(axis=0)
    centroid_B = B.mean(axis=0)

    AA = A - centroid_A
    BB = B - centroid_B

    H = AA.T @ BB

    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = Vt.T @ U.T

    # scale factor
    var_A = (AA ** 2).sum()
    s = (S.sum()) / var_A

    t = centroid_B - s * R @ centroid_A

    return s, R, t



p_tcp = np.array([
    [0.2000, -0.1000, 0.2500],
    [0.2925, -0.0651, 0.2346],
    [0.2361,  0.0378, 0.2891],
])


p_world = np.array([
    [  0,   0, 0],
    [ 10,   0, 0],
    [ 10,  40, 0],
])

s, R_w2tcp, t_w2tcp = similarity_transform_3D(p_world, p_tcp)

# Convert world -> tcp
def world_to_tcp(p):
    return s * ( R_w2tcp @ p ) + t_w2tcp

p_tcp = np.array([
    [0.2000, -0.1000, 0.2500],
    [0.2925, -0.0651, 0.2346],
    [0.2361,  0.0378, 0.2891],
    [0.1436,  0.0029, 0.3045],
    [0.2462, -0.0824, 0.2423],
    [0.2181, -0.0312, 0.2701],
    [0.2643, -0.0136, 0.2618],
])

p_world = np.array([
    [  0,   0, 0],
    [ 10,   0, 0],
    [ 10,  40, 0],
    [  0,  40, 0],
    [  5,   0, 0],
    [  5,  20, 0],
    [ 10,  20, 0],
])

# test: transform tcp[i] -> world and compare with p_world[i]
for i in range(len(p_tcp)):
    predicted = world_to_tcp(p_world[i])
    print("pred:", predicted, " real:", p_tcp[i])
    if not np.allclose(predicted, p_tcp[i], atol=1e-6):
        print("Mismatch at index", i)

motion = []
for x,y,z in p_tcp:
    t = TCP6D.createFromMetersRadians(x,y,z,0,3.14,0)
    m = TCP6DDescriptor(t, t=3)
    motion.append(m)

robot_arm.movej(j_o)
robot_arm.movel_waypoints(motion)
robot_arm.movej(j_o)

## Test motion

It is possible to store robot position in two ways;

1. Tool Center Position(TCP) Position (TCP6D);
    - This position is center on the tool position by encoding the cartesian coordinate (**x**,**y**,**z**) and the orientation along the axis (**rx**,**ry**,**rz**) of the tool.
    - **warning**: Will it make easy to compute the tool position, the robot may have multiple way to reach the say tool position.
    - *Application*: When we need to know the tool position without error.

2. Joint Position (Joint6D);
    - A robot arm is composed of different motor called joint. By setting each of these robot to a specific position the robot can reach a position. In this configuration the robot will have only one possibility to reach the position as it encode the rotation of each joint of the robot.
    - **Warning**: Will it make the position of the robot know without any doubes, it is more difficult to know the end point of the robot -> TCP
    - *Application*: For safe position, and calibration position.



In [ ]:
""" 
movel(self, pose : TCP6D, a : float = 1.2, v : float = 0.25, t : float = 0, r : float = 0, wait : bool = True)
    Move to position (linear in tool-space)
    Parameters:
    pose: target pose [m]
    a:    tool acceleration [m/s^2]
    v:    tool speed [m/s]
    t:    time [S]
    r:    blend radius [m]
    wait: function return when movement is finished

movej(self, joints : Joint6D, a : float = 1.4, v : float = 1.05, t : float = 0, r : float = 0, wait : bool = True)
    Move to position (linear in joint-space)
    Parameters:
    joints:    joint positions [rad]
    a:    joint acceleration of leading axis [rad/s^2]
    v:    joint speed of leading axis [rad/s]
    t:    time [S]
    r:    blend radius [m]
    wait: function return when movement is finished
"""

j_o = Joint6D.createFromRadians(
        -1.5708, -1.5708, -1.0472, -2.0944, 1.5708, 0.0
        # θ_1,    θ_2,     θ_3,     θ_4,    θ_5,    θ_6
)

motion = [
    # 1. Move robot to a safe home position (joint space)
    j_o,

    # 2. Move above the pick location (TCP space)
    TCP6D.createFromMetersRadians(
        0.200, -0.250, 0.300,    # x, y, z in meters
        3.1416, 0.0000, 0.0000   # rx, ry, rz in radians
    ),

    # 3. Move down to the pick point (TCP space)
    TCP6D.createFromMetersRadians(
        0.200, -0.250, 0.180,
        3.1416, 0.0000, 0.0000
    ),

    # 4. Lift back up (TCP space)
    TCP6D.createFromMetersRadians(
        0.200, -0.250, 0.300,
        3.1416, 0.0000, 0.0000
    ),

    # 5. Move to a different joint configuration (joint space)
    Joint6D.createFromRadians(
        1.5708, -1.0472, 1.0472, -1.0472, 1.5708, 1.5708
    ),

    # 6. Move to a place location (TCP space)
    TCP6D.createFromMetersRadians(
        -0.200, -0.250, 0.350,
        3.1416, 0.0000, 0.0000
    ),

    # 7. Return to a safe home position (joint space)
    j_o,
]

for m in motion:
    if isinstance(m, TCP6D):
        robot_arm.movel(m)
    elif isinstance(m, Joint6D):
        robot_arm.movej(m)

## Test gripper

**Important**: Before using the gripper it should be activated.

**Important**: When the gripper will not be used you can deactivat it.

In [ ]:
robot_gripper = iscoin.gripper

robot_gripper.activate()

robot_gripper.waitUntilActive()
if not robot_gripper.isActivated():
  print("Error the gripper was not activated")

In [ ]:
print("Actual gripper opening:\t" + str(robot_opened_gripper_size_mm) + " [mm]")
print("Approximate resolution:\t" + str(robot_opened_gripper_size_mm/255) + " [mm] per unit")

Actual gripper opening:	40.0 [mm]
Approximate resolution:	0.1568627450980392 [mm] per unit


Actual gripper opening: 40 mm  
Approximate resolution: ~0.2 mm per unit

### Basic Command Gripper

In [ ]:
"""
  pos: Position to move to (0-255, 0 being open (50mm), 255 being closed (0mm)). Around 0.2mm per unit.
  speed: Speed of the gripper (0-255, 0 being slowest) default = 255
  force: Force of the gripper (0-255, 0 being weakest) default = 50
"""

if not robot_gripper.move(pos=155, speed=255, force=50): # 20mm -> 255 - (20/0.2) = 155
  print("Failed to move gripper")
sleep(1) # Allow time for the gripper to move

if not robot_gripper.isMoving():
  print("The gripper is still moving")
  robot_gripper.waitUntilStopped() # Alternative to the sleep

if not robot_gripper.open(speed=255, force=50):
  print("Failed to open gripper")
else:
  robot_gripper.waitUntilStopped()
  if not robot_gripper.close(speed=255, force=50):
    print("Failed to close gripper")
robot_gripper.waitUntilStopped()

### Gripper Object Detection

In [ ]:
robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

if not robot_gripper.hasDetectedObject():
    print("The gripper did not catch an object")
else:
    print("The gripper did catch an object")

if robot_gripper.isMoving():
    print("I cannot measure the object dimension as the gripper move.")
else:
    print("The gripper catch an object of dimension " + str(robot_gripper.getEstimatedPositionMm())) + "mm")

sleep(10)

robot_gripper.open(speed=255, force=50)
robot_gripper.waitUntilStopped()

# This will close the gripper and say if the object match the expectations
if not robot_gripper.closeAndCheckSize(expected_size_mm=15., plus_margin_mm=2.5, minus_margin_mm=3.2, speed=255, force=50):
    print("The gripper did not catch the expected object:     " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
else:
    print("The gripper did catch a potential expected object: " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
# robot_gripper.waitUnitilStopped() is incorporated into closeAndCheckSize function

sleep(10)

robot_gripper.open(speed=255, force=50)
robot_gripper.waitUntilStopped()
robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

# This will open the gripper and say if the object match the expectations
if not robot_gripper.openAndCheckSize(expected_size_mm=15., plus_margin_mm=2.5, minus_margin_mm=3.2, speed=255, force=50)
    print("The gripper did not catch the expected object:     " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
else:
    print("The gripper did catch a potential expected object: " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
# robot_gripper.waitUnitilStopped() is incorporated into openAndCheckSize function

sleep(10)

robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

In [ ]:
robot_gripper.move(pos=55, speed=255, force=50) # 40mm -> 255 - (40/0.2) = 55
robot_gripper.waitUntilStopped()
robot_gripper.deactivate()

## Camera

In [ ]:
iscoin.camera.resetCameraSettings()
iscoin.displayCameraStreamOCVParallel() # Display the camera stream

In [ ]:
import cv2
from PIL import Image

from URBasic import CameraSettings
from URBasic import RobotiqWristCamera

In [ ]:
# Plays with camera LEDs
robot_camera = iscoin.camera
cs = CameraSettings()

for i in range(0,11):
    print(f'Setting manual mode to {10*i}')
    cs.lightingSettings.setManualMode(10*i)
    # Actually send changes
    if not robot_camera.setCameraSettings(cs):
        print('Error setting camera settings')
        break
    sleep(0.2)
print('Reset default mode')
cs.lightingSettings.setOffMode()
robot_camera.setCameraSettings(cs)
sleep(3)

img = robot_camera.getImageAsImage(type = RobotiqWristCamera.ImageType.COLOR)

cv2.sho